# Lie Algebras calculation in Python

In [3]:
from scipy.spatial.transform import Rotation as R
import numpy as np

### 基本使用：创建旋转对象，叠加，作用

In [ ]:
# 旋转向量
rot_vec = [0,0,np.pi/2]
# 旋转矩阵
rot_matrix = np.array([[0,-1,0],[1,0,0],[0,0,1]])
# 四元数，默认为[x, y, z, w] 示例为绕z旋转90度
rot_quaternion = [0, 0, np.sin(np.pi/4), np.cos(np.pi/4)]

# 获得旋转的对象
r1 = R.from_rotvec(rot_vec)
r2 = R.from_matrix(rot_matrix)
r3 = R.from_quat(rot_quaternion)

# 多个旋转的叠加，实际效果为绕z转180度
r = r3 * r1

test_vec = (1,0,0)

# apply作用此旋转
after_rot = r.apply(test_vec)

print(np.round(after_rot,3))

[-1.  0.  0.]


### 求逆旋转

In [47]:
r_inv = r.inv()

print('Identical verification')
print((r_inv * r).as_matrix(),end='\n\n')

inversed = r_inv.apply(after_rot)
print(np.round(inversed,3))

Identical verification
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

[1. 0. 0.]


### 不同旋转算符的转换

In [51]:
# 使用as_matrix()把其他表示改变为旋转矩阵
rot_matrix = r3.as_matrix()
print(f"rot_mat: \n{np.round(rot_matrix,2)}",end='\n\n')
# 使用as_quat()把其他表示改变为四元数
rot_quat = r1.as_quat()
print(f"rot_quat: \n{np.round(rot_quat,2)}",end='\n\n')
# 使用as_euler('zyx')转换为欧拉角
# 我们知道zyx欧拉角即为xyz固定角，于是此方法得到的为xyz固定角的结果
rot_euler = r1.as_euler('zyx', degrees=True)
# 使用大写字母时为外旋：绕世界坐标系即固定角
rot_euler = r1.as_euler('XYZ', degrees=True)
# 如下可见两种方式的计算结果是相同的
print(f"rot_euler ('zyx', degrees): \n{np.round(rot_euler, 1)}")
print(f"rot_euler ('XYZ', degrees): \n{np.round(rot_euler, 1)}")

rot_mat: 
[[ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]

rot_quat: 
[0.   0.   0.71 0.71]

rot_euler ('zyx', degrees): 
[-0.  0. 90.]
rot_euler ('XYZ', degrees): 
[-0.  0. 90.]


## Use `pylie` module
Concentrate on handling SE(3), se(3), SO(3) and so(3)

好的，我们来详细了解一下 **`pylie`** 这个 Python 库。

`pylie` 是一个专门为 Python 设计的、用于处理 **李群 (Lie Group)** 和 **李代数 (Lie Algebra)** 的库。它的核心目标是提供一个简洁、易用的接口来表示和操作在机器人学、计算机视觉（尤其是 SLAM、三维重建）和状态估计中非常重要的几何变换，特别是 **三维旋转 SO(3)** 和 **三维刚体变换（位姿） SE(3)**。

`pylie` 库正是围绕上述概念，为 SO(3) 和 SE(3) 提供了 Python 类和方法：

1.  **核心类:**
    *   `pylie.SO3`: 用于表示和操作三维旋转。
    *   `pylie.SE3`: 用于表示和操作三维刚体变换（位姿）。
    *   （可能也包含二维的 `SO2`, `SE2`，具体看库的实现）
2.  **创建对象:** 可以从多种格式创建 `SO3` 和 `SE3` 对象，例如：
    *   `SO3.from_matrix(rotation_matrix)`
    *   `SO3.from_quaternion(quat_xyzw)` # 注意 `pylie` 通常用 [x, y, z, w] 格式
    *   `SE3(rotation, translation)`: 用一个 `SO3` 对象和 NumPy 数组表示的平移向量构造。
    *   `SE3.from_matrix(transformation_matrix)`: 从 4x4 齐次矩阵创建。
3.  **核心操作 (与李理论紧密相关):**
    *   **`exp(algebra_vector)`**: 类方法，执行指数映射。例如 `SO3.exp(so3_vec)` 或 `SE3.exp(se3_vec)`。
    *   **`log()`**: 实例方法，执行对数映射。例如 `so3_obj.log()` 或 `se3_obj.log()`。
    *   **`inverse()`**: 计算逆变换。
    *   **`__mul__(other)` (即 `*` 运算符)**: 实现变换的复合。例如 `T2 * T1` 表示先应用 T1 再应用 T2。
    *   **`act(point)`**: 将变换作用于一个点（通常是 NumPy 数组）。`T.act(p)`。
    *   **`adjoint()`**: 计算伴随矩阵 (Adjoint)，用于在不同坐标系下转换李代数表示。
    *   **雅可比矩阵 (Jacobians)**: `pylie` 通常也提供计算 `exp`, `log` 以及 `act` 操作相对于状态或输入的雅可比矩阵的功能，这对于基于梯度的优化至关重要。
4.  **与 NumPy 的集成:** 底层通常使用 NumPy 进行向量和矩阵运算，方便与 Python 生态中的其他科学计算库配合。

In [11]:
import pylie

In [2]:
# 1. 创建 SO3 对象
# (a) 从旋转向量 (李代数 so(3)) 通过 exp 映射
import pylie.hmanifold
import numpy as np

so3_vec = np.array([0.1, -0.2, np.pi/2]) # 一个 so(3) 向量
R1 = pylie.SO3.exp(so3_vec)
print(f"R1 (from exp): {R1}") # 打印内部表示，通常是四元数

# (b) 从单位四元数 [x, y, z, w]
quat = np.array([0, 0, np.sin(np.pi/4), np.cos(np.pi/4)]) # 绕 Z 轴 90 度
R2 = pylie.SO3.from_quaternion(quat)
print(f"R2 (from quat): {R2}")



AttributeError: module 'pylie' has no attribute 'SO3'